# Outline

This notebook is for comparing the dynamical and stochastic models from scVelo

In [3]:
import anndata
import scvelo as scv
import pandas as pd
import numpy as np
import matplotlib as plt
import igraph
from matplotlib import pyplot
from matplotlib.colors import ListedColormap
import scanpy as sc
import os
import re

In [14]:
def velo_len_conf_table(ob):
    keys = ["velocity_length", "velocity_confidence"]
    # get dataframe with conf values for each object
    stoc_df = stochastic_dict[ob].obs.groupby("Ann_Level3")[keys].mean().T. \
        rename(index = {"velocity_length": "stoc_velo_length", "velocity_confidence": "stoc_velo_confidence"})
    dyn_df = dynamical_dict[ob].obs.groupby("Ann_Level3")[keys].mean().T. \
        rename(index = {"velocity_length": "dyn_velo_length", "velocity_confidence": "dyn_velo_confidence"})
    dyn_df.reindex(index = ["sn_velo_length", "sn_velo_confidence"])
    comb_df = pd.concat([stoc_df, dyn_df])
    styled_out = comb_df.style.background_gradient(subset=pd.IndexSlice[["stoc_velo_length", "dyn_velo_length"], :], axis=None, cmap = "coolwarm") \
        .background_gradient(subset=pd.IndexSlice[["stoc_velo_confidence", "dyn_velo_confidence"], :], axis=None, cmap = "coolwarm") \
        .set_caption(ob) \
        .set_table_styles([{
            "selector": "caption",
            "props": [
                ("color", "black"),
                ("font-size", "24px")
            ]
        }])
    display(styled_out)
    # html = styled_out.to_html()
    # imgkit.from_string(html, "output/figures/rna_velocity/confidence_tables/" + ob + ".html")


In [ ]:
sc_obs = os.listdir("loom_output/split_ad/sc/")
stochastic_dict = dict()
dynamical_dict = dict()
for ob in sc_obs:
    ob = ob.removesuffix(".ad")
    stochastic_dict[ob] = anndata.read(
        "loom_output/split_ad/stochastic/sc/" + ob + ".ad"
    )
    dynamical_dict[ob] = anndata.read(
        "loom_output/split_ad/sc/" + ob + ".ad"
    )
    # add confidence scores
    scv.tl.velocity_confidence(stochastic_dict[ob])
    scv.tl.velocity_confidence(dynamical_dict[ob])


In [15]:
for ob in sc_obs:
    velo_len_conf_table(ob.removesuffix(".ad"))

Ann_Level3,Basal_Progenitor,Fibrogenic,Interactive,MP_Progenitor,Proliferative,Stressed
stoc_velo_length,20.012659,15.777890,12.092381,13.501425,13.284568,13.439631
stoc_velo_confidence,0.936012,0.946372,0.935929,0.948598,0.938730,0.926402
dyn_velo_length,9.203981,10.183366,11.252257,9.313611,9.178689,8.471566
dyn_velo_confidence,0.843178,0.905337,0.835606,0.864366,0.839789,0.858316


Ann_Level3,Basal_Progenitor,Fibrogenic,Interactive,MP_Progenitor,Proliferative
stoc_velo_length,4.082716,3.637126,4.266383,3.369626,2.813723
stoc_velo_confidence,0.815302,0.861374,0.753139,0.813231,0.860038
dyn_velo_length,8.960309,8.458953,18.132796,11.452361,7.882037
dyn_velo_confidence,0.784894,0.794441,0.755467,0.768335,0.788299


Ann_Level3,Basal_Progenitor,Fibrogenic,Interactive,MP_Progenitor,Proliferative,Stressed
stoc_velo_length,21.908278,24.096004,23.901051,29.233267,22.997066,38.659061
stoc_velo_confidence,0.920957,0.937086,0.921035,0.988005,0.951020,0.994823
dyn_velo_length,10.679317,17.168278,12.568219,12.106147,13.755095,15.200887
dyn_velo_confidence,0.890246,0.890116,0.849397,0.958644,0.917119,0.896033


Ann_Level3,Basal_Progenitor,Fibrogenic,Interactive,MP_Progenitor,Proliferative,Stressed
stoc_velo_length,6.691882,11.089850,7.431649,7.245794,6.400449,15.563342
stoc_velo_confidence,0.805700,0.924154,0.803165,0.872640,0.873153,0.896541
dyn_velo_length,16.793640,17.250894,16.520220,17.937316,18.259323,19.312235
dyn_velo_confidence,0.824746,0.837447,0.764694,0.808967,0.806581,0.851518


Ann_Level3,Basal_Progenitor,Fibrogenic,Interactive,MP_Progenitor,Proliferative,Stressed
stoc_velo_length,7.676345,7.918671,6.159797,7.195680,6.117892,14.930758
stoc_velo_confidence,0.864718,0.884885,0.856805,0.845412,0.882290,0.919016
dyn_velo_length,20.504738,18.419829,19.064087,20.482563,17.801985,18.307515
dyn_velo_confidence,0.768798,0.794569,0.794458,0.797385,0.830404,0.818633


Ann_Level3,Basal_Progenitor,Fibrogenic,Interactive,MP_Progenitor,Proliferative,Stressed
stoc_velo_length,22.220240,20.892887,17.585543,20.746580,18.382046,23.775307
stoc_velo_confidence,0.928191,0.932236,0.918721,0.928197,0.942457,0.924931
dyn_velo_length,13.981037,15.563418,15.418540,16.323490,14.767128,14.995782
dyn_velo_confidence,0.836151,0.854259,0.817834,0.853706,0.826486,0.831434


Ann_Level3,Basal_Progenitor,Fibrogenic,Interactive,MP_Progenitor,Proliferative,Stressed
stoc_velo_length,12.094235,13.930037,12.327866,9.112852,8.869938,11.532619
stoc_velo_confidence,0.915299,0.931876,0.928600,0.878403,0.920583,0.882841
dyn_velo_length,19.440687,24.379724,22.457171,23.041871,21.633948,27.166331
dyn_velo_confidence,0.719819,0.838596,0.802840,0.791671,0.790473,0.889079


Ann_Level3,Basal_Progenitor,Fibrogenic,Interactive,MP_Progenitor,Proliferative
stoc_velo_length,11.657218,34.990894,19.113609,55.184319,14.107695
stoc_velo_confidence,0.874328,0.877933,0.827442,0.887896,0.877611
dyn_velo_length,38.968111,42.570357,55.351353,58.062952,41.414002
dyn_velo_confidence,0.715287,0.832739,0.771178,0.806069,0.707946
